In [ ]:
pip install deepface tf-keras opencv-python

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from deepface import DeepFace
import cv2
import matplotlib.pyplot as plt

def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const video = document.createElement('video');
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await video.play();

            await new Promise(resolve => setTimeout(resolve, 2000));

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);

            stream.getTracks().forEach(track => track.stop());

            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)
    data = eval_js(f'takePhoto({quality})')
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)

take_photo()

img = cv2.imread('photo.jpg')
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

results = DeepFace.analyze(
    img_path=img_rgb,
    actions=['emotion'],
    enforce_detection=False
)

# Handle list output
if isinstance(results, list):
    res = results[0]

# Extract values for the specific output format
emotion = res['dominant_emotion']
confidence = res['emotion'][emotion]

# --- THE SPECIFIC OUTPUT FORMAT YOU REQUESTED ---
print(f"Dominant emotion: {emotion}")
print(f"Confidence: {confidence:.2f}%")
print(f"Detected: {emotion}")
# ------------------------------------------------

plt.imshow(img_rgb)
plt.axis('off')
plt.title(f"Emotion: {emotion}")
plt.show()